## Description

We follow the works on distributed GPs and on stochastic GPs which run benchmarks on a huge dataset of taxi rides in the city of New York.

The dataset is not well standardized so some preprocessing steps are necessary. Useful links are:
 - https://github.com/toddwschneider/nyc-taxi-data -- for data download scripts and preprocessing help
 - ? for data description
 - ? first usage? of the dataset
 
The goal of this benchmark is to predict the duration of a trip, given 9 features:
 1. time of the day
 2. day of the week
 3. day of the month
 4. month
 5. pick-up latitude
 6. pick-up longitude
 7. drop-off latitude
 8. drop-off longitude
 9. travel distance
 
While the data is available from 2009 to today, we will only use the trips from January 2009 to December 2015 in order to provide a fair comparison with other methods.

Furthermore, we must exclude trips that are outside the NYC area, and those that are more than 5 hours long.

If all goes well we should have the following aggregate statistics for our target variable:
 - average traveling time is 764 seconds
 - standard derivation is 576 seconds.

After the import into postgresql we have table `trips` with the following columns:
```
id
pickup_datetime
dropoff_datetime
pickup_longitude
pickup_latitude
dropoff_longitude
dropoff_latitude
trip_distance
pickup_nyct2010_gid
dropoff_nyct2010_gid
```

Since we create the git fields by intersecting with a NYC map, the trips where gid is null are assumed to be outside of nyc.

We can count approximately 32M such points

```sql
SELECT COUNT(*) FROM trips WHERE (pickup_nyct2010_gid is null) OR (dropoff_nyct2010_gid is null);
```
```
32'771'382
```

Further, there are a bunch of outliers on the trip duration column: trips with negative duration or trips with extremely long duration.
We exclude all trips more than 5 hours (3600*5=18000) long, and all those with negative duration.

```sql
SELECT COUNT(*) FROM trips WHERE
    EXTRACT(EPOCH FROM dropoff_datetime - pickup_datetime)
    NOT BETWEEN 0 AND 18000;
```
```
481135
```

Excluding both kinds of data-points we get the following summary statistics on the duration column:
```sql
SELECT avg(EXTRACT(EPOCH FROM dropoff_datetime - pickup_datetime)), 
       stddev_samp(EXTRACT(EPOCH FROM dropoff_datetime - pickup_datetime)),
       stddev_pop(EXTRACT(EPOCH FROM dropoff_datetime - pickup_datetime)) 
FROM trips WHERE 
(pickup_nyct2010_gid IS NOT NULL) AND 
(dropoff_nyct2010_gid IS NOT NULL) AND
(EXTRACT(EPOCH FROM dropoff_datetime - pickup_datetime) BETWEEN 0 AND 18000);
```
```sql
       avg        |   stddev_samp    |    stddev_pop    
------------------+------------------+------------------
 752.520099079508 | 566.116904461205 | 566.116904214329
```

Copy to CSV using pagination

```sql
COPY (
    SELECT
        id,
        EXTRACT(EPOCH FROM CAST(pickup_datetime AS time)) as time,
        EXTRACT(ISODOW FROM pickup_datetime) as dow,
        EXTRACT(DAY FROM pickup_datetime) as dom,
        EXTRACT(MONTH FROM pickup_datetime) as month,
        round(pickup_latitude, 6),
        round(pickup_longitude, 6),
        round(dropoff_latitude, 6),
        round(dropoff_longitude, 6),
        round(trip_distance, 3),
        EXTRACT(EPOCH FROM dropoff_datetime - pickup_datetime) as duration
    FROM trips WHERE 
        (pickup_nyct2010_gid IS NOT NULL) AND 
        (dropoff_nyct2010_gid IS NOT NULL) AND
        (EXTRACT(EPOCH FROM dropoff_datetime - pickup_datetime) BETWEEN 0 AND 18000) AND
        (id > 13385191)
    ORDER BY id ASC
    LIMIT 10
) TO STDOUT
WITH (FORMAT csv, HEADER true);
```

In [46]:
import psycopg2 
import sys, os, time

In [47]:
csv_size = 3_000_000
folder = "/data/DATASETS/NYCTAXI/PROCESSED/"
try:
    os.makedirs(folder)
except FileExistsError:
    pass

In [48]:
def make_copy_sql(from_id, num_rows, out_file):
    return """
    COPY (
        SELECT
            id,
            EXTRACT(EPOCH FROM CAST(pickup_datetime AS time)) as time,
            EXTRACT(ISODOW FROM pickup_datetime) as dow,
            EXTRACT(DAY FROM pickup_datetime) as dom,
            EXTRACT(MONTH FROM pickup_datetime) as month,
            round(pickup_latitude, 6) as pickup_lat,
            round(pickup_longitude, 6) as pickup_lon,
            round(dropoff_latitude, 6) as dropoff_lat,
            round(dropoff_longitude, 6) as dropoff_lon,
            round(trip_distance, 3) as distance,
            EXTRACT(EPOCH FROM dropoff_datetime - pickup_datetime) as duration
        FROM trips WHERE 
            (pickup_nyct2010_gid IS NOT NULL) AND 
            (dropoff_nyct2010_gid IS NOT NULL) AND
            (EXTRACT(EPOCH FROM dropoff_datetime - pickup_datetime) BETWEEN 0 AND 18000) AND
            (id > %d)
        ORDER BY id ASC
        LIMIT %d
    ) TO '%s'
    WITH (FORMAT csv, HEADER true);""" % (from_id, num_rows, out_file)

In [49]:
conn.close()
conn = psycopg2.connect(
    host="localhost", 
    port=5432, 
    database="nyc-taxi-data", 
    user="giacomo")

In [50]:
index = 0
i = 0
with conn.cursor() as cursor:
    while True:
        t_s = time.time()
        fn = os.path.join(folder, "%d.csv" % (index))
        # Postgres server needs to have permission to create the file
        os.umask(0)
        with open(os.open(fn, os.O_CREAT | os.O_WRONLY, 0o777), 'w') as fh:
            cursor.copy_expert(make_copy_sql(index, csv_size, fn), fh)
        # Read last line of written file to check the new index
        last_line = os.popen('tail -n 1 %s' % (fn)).read()
        if last_line == "":
            break
        index = int(last_line.split(",")[0])
        # Compress the file
        os.popen('gzip -f %s' % (fn))

        i += 1
        print("%d - %.2fs - Retrieved new start ID %d" % (i, time.time() - t_s, index))

1 - 14.19s - Retrieved new start ID 16500092
2 - 15.10s - Retrieved new start ID 19614671
3 - 15.00s - Retrieved new start ID 22730160
4 - 16.37s - Retrieved new start ID 25844686
5 - 16.35s - Retrieved new start ID 28944407
6 - 15.43s - Retrieved new start ID 32027576
7 - 15.66s - Retrieved new start ID 35110928
8 - 15.70s - Retrieved new start ID 38193871
9 - 15.40s - Retrieved new start ID 41275223
10 - 15.39s - Retrieved new start ID 44347304
11 - 16.78s - Retrieved new start ID 47418586
12 - 17.12s - Retrieved new start ID 50489809
13 - 15.37s - Retrieved new start ID 53561398
14 - 16.45s - Retrieved new start ID 56634818
15 - 16.23s - Retrieved new start ID 59710543
16 - 16.26s - Retrieved new start ID 62785891
17 - 15.85s - Retrieved new start ID 65861415
18 - 15.57s - Retrieved new start ID 68936775
19 - 15.78s - Retrieved new start ID 72013909
20 - 16.09s - Retrieved new start ID 75090936
21 - 15.21s - Retrieved new start ID 78167466
22 - 16.03s - Retrieved new start ID 812442

175 - 15.53s - Retrieved new start ID 554706500
176 - 15.55s - Retrieved new start ID 557806295
177 - 15.63s - Retrieved new start ID 560905635
178 - 15.71s - Retrieved new start ID 564002866
179 - 15.74s - Retrieved new start ID 567101460
180 - 15.98s - Retrieved new start ID 570200007
181 - 15.31s - Retrieved new start ID 573298160
182 - 15.61s - Retrieved new start ID 576396154
183 - 15.79s - Retrieved new start ID 579494476
184 - 16.90s - Retrieved new start ID 582593228
185 - 15.13s - Retrieved new start ID 585692011
186 - 15.50s - Retrieved new start ID 588790539
187 - 16.10s - Retrieved new start ID 591889373
188 - 15.72s - Retrieved new start ID 594994528
189 - 15.02s - Retrieved new start ID 598099417
190 - 15.40s - Retrieved new start ID 601204396
191 - 15.83s - Retrieved new start ID 604309650
192 - 15.83s - Retrieved new start ID 607414761
193 - 16.25s - Retrieved new start ID 610520601
194 - 16.08s - Retrieved new start ID 613626317
195 - 16.07s - Retrieved new start ID 61

346 - 15.77s - Retrieved new start ID 1082121418
347 - 16.12s - Retrieved new start ID 1085199930
348 - 15.72s - Retrieved new start ID 1088272650
349 - 16.21s - Retrieved new start ID 1091344207
350 - 15.88s - Retrieved new start ID 1094415113
351 - 15.75s - Retrieved new start ID 1097487252
352 - 16.96s - Retrieved new start ID 1100556542
353 - 16.14s - Retrieved new start ID 1103625009
354 - 15.05s - Retrieved new start ID 1106693976
355 - 16.15s - Retrieved new start ID 1109761923
356 - 15.66s - Retrieved new start ID 1112830058
357 - 16.68s - Retrieved new start ID 1115895600
358 - 17.21s - Retrieved new start ID 1118961265
359 - 15.97s - Retrieved new start ID 1122027125
360 - 15.82s - Retrieved new start ID 1125092153
361 - 17.13s - Retrieved new start ID 1128155997
362 - 16.09s - Retrieved new start ID 1131220057
363 - 15.73s - Retrieved new start ID 1134284289
364 - 16.90s - Retrieved new start ID 1137347802
365 - 16.49s - Retrieved new start ID 1140411152
366 - 16.26s - Retri

ValueError: invalid literal for int() with base 10: 'id'

# Preprocessing and Analysis

### Changing the file format
The data is in zipped csv files that are super slow to load.
We convert it to a single h5py dataset, compressed, with the following keys:
 - X : the data matrix
 - Y : the targets

In [9]:
import numpy
import torch
import pandas as pd
import h5py
import time
import os
import numpy as np

In [10]:
data_folder = "/data/DATASETS/NYCTAXI/PROCESSED/"
def list_files(folder):
    for r, d, f in os.walk(folder):
        for file in f:
            if file.endswith('.csv.gz'):
                yield os.path.join(r, file)

First we read the data in memory, and place it into a separate numpy array per each file

In [11]:
all_x = []
all_y = []
for f in list_files(data_folder):
    t_s = time.time()
    df = pd.read_csv(f, header=0, index_col=False)
    Y = df['duration'].to_numpy(np.int32, copy=True)
    X = df[['time', 'dow', 'dom', 'month', 'round',
            'round.1', 'round.2', 'round.3',
            'round.4']].to_numpy(np.float64, copy=True)
    all_x.append(X)
    all_y.append(Y)
    del df
    print("file %s read in %.2fs" % (f, time.time() - t_s))

file /data/DATASETS/NYCTAXI/PROCESSED/1066724750.csv.gz read in 5.11s
file /data/DATASETS/NYCTAXI/PROCESSED/616730994.csv.gz read in 4.70s
file /data/DATASETS/NYCTAXI/PROCESSED/700318523.csv.gz read in 4.59s
file /data/DATASETS/NYCTAXI/PROCESSED/294064714.csv.gz read in 4.72s
file /data/DATASETS/NYCTAXI/PROCESSED/1002169437.csv.gz read in 4.76s
file /data/DATASETS/NYCTAXI/PROCESSED/359154558.csv.gz read in 4.66s
file /data/DATASETS/NYCTAXI/PROCESSED/340566875.csv.gz read in 4.66s
file /data/DATASETS/NYCTAXI/PROCESSED/1014456771.csv.gz read in 4.75s
file /data/DATASETS/NYCTAXI/PROCESSED/755670561.csv.gz read in 4.59s
file /data/DATASETS/NYCTAXI/PROCESSED/105750702.csv.gz read in 4.82s
file /data/DATASETS/NYCTAXI/PROCESSED/38193871.csv.gz read in 4.66s
file /data/DATASETS/NYCTAXI/PROCESSED/349859862.csv.gz read in 4.70s
file /data/DATASETS/NYCTAXI/PROCESSED/207237975.csv.gz read in 4.72s
file /data/DATASETS/NYCTAXI/PROCESSED/996008571.csv.gz read in 4.60s
file /data/DATASETS/NYCTAXI/PROC

file /data/DATASETS/NYCTAXI/PROCESSED/1137347802.csv.gz read in 5.06s
file /data/DATASETS/NYCTAXI/PROCESSED/418305284.csv.gz read in 4.92s
file /data/DATASETS/NYCTAXI/PROCESSED/1115895600.csv.gz read in 5.05s
file /data/DATASETS/NYCTAXI/PROCESSED/176415963.csv.gz read in 5.10s
file /data/DATASETS/NYCTAXI/PROCESSED/216498298.csv.gz read in 4.96s
file /data/DATASETS/NYCTAXI/PROCESSED/225774499.csv.gz read in 5.12s
file /data/DATASETS/NYCTAXI/PROCESSED/647802973.csv.gz read in 4.94s
file /data/DATASETS/NYCTAXI/PROCESSED/222676062.csv.gz read in 5.34s
file /data/DATASETS/NYCTAXI/PROCESSED/290953966.csv.gz read in 4.97s
file /data/DATASETS/NYCTAXI/PROCESSED/983609483.csv.gz read in 5.01s
file /data/DATASETS/NYCTAXI/PROCESSED/632259696.csv.gz read in 4.92s
file /data/DATASETS/NYCTAXI/PROCESSED/325060094.csv.gz read in 5.06s
file /data/DATASETS/NYCTAXI/PROCESSED/28944407.csv.gz read in 5.00s
file /data/DATASETS/NYCTAXI/PROCESSED/281623287.csv.gz read in 5.09s
file /data/DATASETS/NYCTAXI/PROCE

file /data/DATASETS/NYCTAXI/PROCESSED/734159387.csv.gz read in 5.44s
file /data/DATASETS/NYCTAXI/PROCESSED/1152666907.csv.gz read in 5.48s
file /data/DATASETS/NYCTAXI/PROCESSED/393455014.csv.gz read in 5.43s
file /data/DATASETS/NYCTAXI/PROCESSED/1029806272.csv.gz read in 5.41s
file /data/DATASETS/NYCTAXI/PROCESSED/377841012.csv.gz read in 5.32s
file /data/DATASETS/NYCTAXI/PROCESSED/415210256.csv.gz read in 5.56s
file /data/DATASETS/NYCTAXI/PROCESSED/427590064.csv.gz read in 5.40s
file /data/DATASETS/NYCTAXI/PROCESSED/145620755.csv.gz read in 5.54s
file /data/DATASETS/NYCTAXI/PROCESSED/961879124.csv.gz read in 5.35s
file /data/DATASETS/NYCTAXI/PROCESSED/573298160.csv.gz read in 5.49s
file /data/DATASETS/NYCTAXI/PROCESSED/1192549527.csv.gz read in 0.95s
file /data/DATASETS/NYCTAXI/PROCESSED/681836491.csv.gz read in 5.49s
file /data/DATASETS/NYCTAXI/PROCESSED/387211204.csv.gz read in 5.38s
file /data/DATASETS/NYCTAXI/PROCESSED/68936775.csv.gz read in 5.43s
file /data/DATASETS/NYCTAXI/PROC

file /data/DATASETS/NYCTAXI/PROCESSED/955673067.csv.gz read in 10.10s
file /data/DATASETS/NYCTAXI/PROCESSED/380967304.csv.gz read in 10.53s
file /data/DATASETS/NYCTAXI/PROCESSED/657115286.csv.gz read in 10.72s
file /data/DATASETS/NYCTAXI/PROCESSED/533018786.csv.gz read in 11.30s
file /data/DATASETS/NYCTAXI/PROCESSED/502042875.csv.gz read in 6.90s
file /data/DATASETS/NYCTAXI/PROCESSED/1039025821.csv.gz read in 10.47s
file /data/DATASETS/NYCTAXI/PROCESSED/560905635.csv.gz read in 10.19s
file /data/DATASETS/NYCTAXI/PROCESSED/539213615.csv.gz read in 10.27s
file /data/DATASETS/NYCTAXI/PROCESSED/903293001.csv.gz read in 5.35s
file /data/DATASETS/NYCTAXI/PROCESSED/1112830058.csv.gz read in 5.62s
file /data/DATASETS/NYCTAXI/PROCESSED/706479150.csv.gz read in 5.28s
file /data/DATASETS/NYCTAXI/PROCESSED/542310012.csv.gz read in 5.31s
file /data/DATASETS/NYCTAXI/PROCESSED/458519885.csv.gz read in 4.98s
file /data/DATASETS/NYCTAXI/PROCESSED/443035983.csv.gz read in 4.93s
file /data/DATASETS/NYCTA

In [12]:
num_samples = sum([arr.shape[0] for arr in all_x])
dim = all_x[0].shape[1]
print(num_samples, ",", dim)

1146561612 , 9


In [13]:
max_chunk_size = 2 * 2**20  # 2MB
chunk_x = int(max_chunk_size / dim / 8)
chunk_y = chunk_x
print("Chunk size:", chunk_x)

Chunk size: 29127


In [14]:
with h5py.File(os.path.join(data_folder, 'full.h5py'), 'w', libver='latest') as f:
    Xdset = f.create_dataset("X", (num_samples, dim), dtype='float64', 
                             compression="gzip", chunks=(chunk_x, dim))
    Ydset = f.create_dataset("Y", (num_samples, 1), dtype='int32')
    current_i = 0
    for X, Y in zip(all_x, all_y):
        t_s = time.time()
        X = np.ascontiguousarray(X)
        Y = Y.reshape((-1, 1))
        Xdset.write_direct(X, dest_sel=np.s_[current_i:current_i+X.shape[0], :])
        Ydset.write_direct(Y, dest_sel=np.s_[current_i:current_i+Y.shape[0], :])
        current_i += X.shape[0]
        print("i: %d/%d in %.2fs" % (current_i, num_samples, time.time() - t_s))

i: 3000000/1146561612 in 7.86s
i: 6000000/1146561612 in 9.08s
i: 9000000/1146561612 in 7.69s
i: 12000000/1146561612 in 7.96s
i: 15000000/1146561612 in 8.49s
i: 18000000/1146561612 in 8.30s
i: 21000000/1146561612 in 9.39s
i: 24000000/1146561612 in 8.55s
i: 27000000/1146561612 in 9.64s
i: 30000000/1146561612 in 9.52s
i: 33000000/1146561612 in 8.54s
i: 36000000/1146561612 in 8.43s
i: 39000000/1146561612 in 9.76s
i: 42000000/1146561612 in 8.20s
i: 45000000/1146561612 in 8.29s
i: 48000000/1146561612 in 8.38s
i: 51000000/1146561612 in 8.38s
i: 54000000/1146561612 in 8.26s
i: 57000000/1146561612 in 8.65s
i: 60000000/1146561612 in 7.63s
i: 63000000/1146561612 in 8.64s
i: 66000000/1146561612 in 8.75s
i: 69000000/1146561612 in 8.89s
i: 72000000/1146561612 in 8.38s
i: 75000000/1146561612 in 8.00s
i: 78000000/1146561612 in 8.43s
i: 81000000/1146561612 in 7.66s
i: 84000000/1146561612 in 7.95s
i: 87000000/1146561612 in 7.93s
i: 90000000/1146561612 in 7.98s
i: 93000000/1146561612 in 8.11s
i: 96000000

i: 750561612/1146561612 in 7.97s
i: 753561612/1146561612 in 10.23s
i: 756561612/1146561612 in 8.37s
i: 759561612/1146561612 in 8.02s
i: 762561612/1146561612 in 7.15s
i: 765561612/1146561612 in 9.14s
i: 768561612/1146561612 in 8.35s
i: 771561612/1146561612 in 7.67s
i: 774561612/1146561612 in 8.13s
i: 777561612/1146561612 in 8.20s
i: 780561612/1146561612 in 8.16s
i: 783561612/1146561612 in 7.71s
i: 786561612/1146561612 in 8.17s
i: 789561612/1146561612 in 8.52s
i: 792561612/1146561612 in 7.73s
i: 795561612/1146561612 in 8.13s
i: 798561612/1146561612 in 8.15s
i: 801561612/1146561612 in 8.48s
i: 804561612/1146561612 in 7.97s
i: 807561612/1146561612 in 8.03s
i: 810561612/1146561612 in 7.71s
i: 813561612/1146561612 in 9.56s
i: 816561612/1146561612 in 7.77s
i: 819561612/1146561612 in 8.16s
i: 822561612/1146561612 in 8.18s
i: 825561612/1146561612 in 9.05s
i: 828561612/1146561612 in 8.02s
i: 831561612/1146561612 in 7.08s
i: 834561612/1146561612 in 7.11s
i: 837561612/1146561612 in 9.50s
i: 840561